<a href="https://colab.research.google.com/github/azile-g/option_pricing/blob/main/OptionPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
from math import floor, exp

In [ ]:
class initialising: 
    def __init__(self, S, K, r, q, T, sigma): 
        self.T = float(T)/12 #expiry time (input in months converted to year)
        self.r = float(r)/100 #risk-free rate (input in percentage converted to dp)
        self.sigma = float(sigma) #volatility 
        self.K = float(K) #Strike price 
        self.q = float(q) #dividend
        self.S = float(S) #underlying stock price

In [ ]:
class Option_Pricing (initialising): 

    def explicit(self, M, N):
        #Step 1: Discretisation 
        S_max = self.K*2
        dt = self.T/N
        ds = S_max/M 
        
        #step 2:
        matrix_call = np.zeros((M+1, N+1)) # Create F_i
        matrix_put = np.zeros((M+1, N+1))
        N_array = np.arange(M+1)
        matrix_call[:,N]= np.maximum(N_array * ds - self.K,0) # Step 2, to compute F(N,j)
        matrix_put[:,N]= np.maximum(self.K - N_array * ds,0)

        #Step 3:
        #Compute matrix A
        A = np.zeros((M+1, M+1))
        A[0, 0] = 1
        for j in range(1, M):
            A[j, j-1] = 0.5*(dt)*(self.sigma**2*j**2-(self.r-self.q)*j)
            A[j, j] = 1-dt*(self.sigma**2*j**2+self.r)
            A[j, j+1] = 0.5*(dt)*(self.sigma**2*j**2+(self.r-self.q)*j)
        A[M, M] = 1

        #Step 4: Compute F_i 
        for i in range(N-1,-1,-1): 
            # Step 3.1 for call
            matrix_call[:, i] =np.dot(A, matrix_call[:, i+1])
            # Can also use matrix_call[:, [i]] = A@ matrix_call[:, [i+1]]  Calculate F_hat_i from i=N-1
            # Step 3.2 for call
            matrix_call[0, i] = 0 # Assign 0 to the first row of F_i            
            matrix_call[-1, i] = S_max-self.K *exp(-self.r*(N-i)*(dt)) # Assign S_max-E*math.exp(-r*(N-i)*(dt)) to the last row of F_i 
            # Step 3.1 for put
            matrix_put[:, i] =np.dot(A, matrix_put[:, i+1])
            # Step 3.2 for put
            matrix_put[-1, i] = 0 # Assign 0 to the first row of F_i            
            matrix_put[0, i] = self.K * exp(-self.r*(N-i)*(dt)) # Assign K*math.exp(-r*(N-i)*(dt)) to the first row of F_i  
            
        #Step 5: 
        #Find k
        k = floor(self.S/ds)

        #Option price 
        call_price = matrix_call[k, 0] + ((matrix_call[k+1,0]-matrix_call[k,0])/ds)*(self.S-k*ds)
        put_price = matrix_put[k, 0] + ((matrix_put[k+1,0]-matrix_put[k,0])/ds)*(self.S-k*ds)

        return {'call':call_price,'put':put_price}